In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from io import StringIO
import os
import textwrap
from io import StringIO
from tabulate import tabulate

In [72]:
def Save_Genreal_Report(corr_matrix, dup, head, tail, des, null, info, shape):
    #  report = input("Enter the file name to save the report (e.g., report.txt): ")
     try:
        folder_path = input("Enter the full path or name of the folder to save the report and heatmap: ").strip()
        os.makedirs(folder_path, exist_ok=True)
        report = os.path.join(folder_path, "report.txt")
        heatmap_file = os.path.join(folder_path, "correlation_heatmap.png")
        plt.figure(figsize=(8, 6))
        plt.imshow(corr_matrix, cmap='coolwarm', interpolation='nearest')
        plt.colorbar()
        plt.xticks(np.arange(len(corr_matrix.columns)), corr_matrix.columns, rotation=45, fontsize=12)
        plt.yticks(np.arange(len(corr_matrix.columns)), corr_matrix.columns, rotation=45, fontsize=12)
        plt.title('Correlation Matrix Heatmap', fontsize=14)
        plt.tight_layout()
        plt.savefig(heatmap_file)  
        plt.close()
        with open(report, 'w') as f:
            f.write("GENERAL ANALYSIS OF THE DATAFRAME\n")
            f.write("*" * 50 + "\n\n")
            f.write("-" * 50 + "\n")
            f.write(f"Shape of DataFrame: {shape}\n")
            f.write("-" * 50 + "\n\n\n")
            f.write("-" * 50 + "\n")
            f.write("Column Information:\n")
            f.write("-" * 50 + "\n")
            f.write(info + "\n") 
            f.write("\n\n\n")
            f.write("-" * 50 + "\n")
            f.write("Missing Values in Each Column:\n")
            f.write("-" * 50 + "\n")
            f.write(str(null) + "\n")
            f.write("\n\n\n")
            f.write("-" * 50 + "\n")
            f.write("Descriptive Statistics (Numerical):\n")
            f.write("-" * 50 + "\n")
            f.write(str(des) + "\n")
            # f.write(tabulate(des, headers='keys', tablefmt='psql'))
            f.write("\n\n\n")
            f.write("-" * 50 + "\n")
            f.write("First 5 Rows of the DataFrame:\n")
            f.write("-" * 50 + "\n")
            f.write(str(head) + "\n")
            f.write("\n\n\n")
            f.write("-" * 50 + "\n")
            f.write("Last 5 Rows of the DataFrame:\n")
            f.write("-" * 50 + "\n")
            f.write(str(tail) + "\n")
            f.write("\n\n\n")
            f.write("-" * 50 + "\n")
            f.write(f"Number of Duplicate Rows: {dup}\n")
            f.write("-" * 50 + "\n\n\n")
            f.write("-" * 50 + "\n")
            f.write("Correlation Matrix (Numerical Columns):\n")
            f.write("-" * 50 + "\n")
            f.write(str(corr_matrix) + "\n")
            f.write("\n\n")
        
        print(f"Report successfully saved as '{report}'.")
     except Exception as e:
        print(f"Error: {e}")



    # try:
    #         # Create the folder path
    #         folder_path = input("Enter the full path or name of the folder to save the report and heatmap: ").strip()
    #         os.makedirs(folder_path, exist_ok=True)
            
    #         # Report and heatmap file paths
    #         report = os.path.join(folder_path, "report.txt")
    #         heatmap_file = os.path.join(folder_path, "correlation_heatmap.png")
            
    #         # Create the heatmap
    #         plt.figure(figsize=(8, 6))
    #         plt.imshow(corr_matrix, cmap='coolwarm', interpolation='nearest')
    #         plt.colorbar()
    #         plt.xticks(np.arange(len(corr_matrix.columns)), corr_matrix.columns, rotation=45, fontsize=12)
    #         plt.yticks(np.arange(len(corr_matrix.columns)), corr_matrix.columns, rotation=45, fontsize=12)
    #         plt.title('Correlation Matrix Heatmap', fontsize=14)
    #         plt.tight_layout()
    #         plt.savefig(heatmap_file)  
    #         plt.close()
            
    #         # Write the report in table format
    #         with open(report, 'w') as f:
    #             f.write("GENERAL ANALYSIS OF THE DATAFRAME\n")
    #             f.write("=" * 80 + "\n\n")
                
    #             # Shape of the DataFrame
    #             f.write("📌 Shape of DataFrame:\n")
    #             f.write("-" * 80 + "\n")
    #             f.write(f"Rows: {shape[0]}, Columns: {shape[1]}\n\n")
                
    #             # Column Information
    #             f.write("📌 Column Information:\n")
    #             f.write("-" * 80 + "\n")
    #             wrapped_info = tabulate([[info]], tablefmt="fancy_grid", stralign="center")
    #             f.write(wrapped_info + "\n\n")
                
    #             # Missing Values
    #             f.write("📌 Missing Values in Each Column:\n")
    #             f.write("-" * 80 + "\n")
    #             f.write(tabulate(null.to_frame(), headers=["Column", "Missing Values"], tablefmt="fancy_grid") + "\n\n")
                
    #             # Descriptive Statistics
    #             f.write("📌 Descriptive Statistics (Numerical):\n")
    #             f.write("-" * 80 + "\n")
    #             f.write(tabulate(des, headers="keys", tablefmt="fancy_grid", showindex=True) + "\n\n")
                
    #             # First 5 Rows
    #             f.write("📌 First 5 Rows of the DataFrame:\n")
    #             f.write("-" * 80 + "\n")
    #             f.write(tabulate(head, headers="keys", tablefmt="fancy_grid", showindex=True) + "\n\n")
                
    #             # Last 5 Rows
    #             f.write("📌 Last 5 Rows of the DataFrame:\n")
    #             f.write("-" * 80 + "\n")
    #             f.write(tabulate(tail, headers="keys", tablefmt="fancy_grid", showindex=True) + "\n\n")
                
    #             # Duplicate Rows
    #             f.write("📌 Number of Duplicate Rows:\n")
    #             f.write("-" * 80 + "\n")
    #             f.write(f"{dup} duplicate rows found.\n\n")
                
    #             # Correlation Matrix
    #             f.write("📌 Correlation Matrix (Numerical Columns):\n")
    #             f.write("-" * 80 + "\n")
    #             f.write(tabulate(corr_matrix, headers="keys", tablefmt="fancy_grid", showindex=True) + "\n\n")
                
    #             f.write("=" * 80 + "\n")
    #             f.write("📄 Report Generated Successfully!\n")
            
    #         print(f"✅ Report saved successfully as '{report}'")

    # except Exception as e:
    #         print(f"Error: {e}")




# def Save_General_Report(corr_matrix, dup, head, tail, des, null, info, shape):
#     folder_path = input('Enter Folder Path: ')
#     os.makedirs(folder_path, exist_ok=True)
#     filename = os.path.join(folder_path, "General_Report.csv")

#     # Combine the information into a single dictionary for the CSV file
#     report_data = {
#         "Shape of DataFrame": [f"Rows: {shape[0]}, Columns: {shape[1]}"],
#         "Column Information": [info],
#         "Missing Values in Each Column": [null.to_string()],
#         "Descriptive Statistics (Numerical)": [des.to_string()],
#         "First 5 Rows of DataFrame": [head.to_string()],
#         "Last 5 Rows of DataFrame": [tail.to_string()],
#         "Number of Duplicate Rows": [dup],
#         "Correlation Matrix": [corr_matrix.to_string()]
#     }

#     # Convert report data to DataFrame
#     report_df = pd.DataFrame(report_data)

#     # Save to CSV
#     report_df.to_csv(filename, index=False)

#     print(f"✅ Report saved successfully as {filename}")
    

In [62]:
# def General_Analysis(df):
#     print("GENERAL ANALYSIS OF THE DATAFRAME")
#     print("*" * 50)
#     print("\n" + "\n")
#     shape=df.shape
#     print("-" * 50)
#     print(f"Shape of DataFrame: {shape}" )
#     print("-" * 50)
#     print("\n" + "\n")
#     print("-" * 50)
#     print("Column Information:")
#     print("-" * 50)
#     buffer = StringIO()
#     df.info(buf=buffer)  # Write the info to the buffer
#     info = buffer.getvalue()
#     print(info)
#     print("\n" + "\n")
#     print("-" * 50)
#     print("Missing Values in Each Column:")
#     print("-" * 50)
#     null=df.isnull().sum()
#     print(null)
#     print("\n" + "\n")

#     numdf = df.select_dtypes(include=['number'])
#     print("-" * 50)
#     print("Descriptive Statistics (Numerical):")
#     print("-" * 50)
#     des=numdf.describe()
#     print(des )
#     print("\n" + "\n")

    
#     print("-" * 50)
#     print("First 5 Rows of the DataFrame:")
#     print("-" * 50)
#     head=df.head()
#     print(head )
#     print("\n" + "\n")

#     print("-" * 50)
#     print("Last 5 Rows of the DataFrame:")
#     print("-" * 50)
#     tail=df.tail()
#     print(tail )
#     print("\n" + "\n")

#     print("-" * 50)
#     dup=df.duplicated().sum()
#     print(f"Number of Duplicate Rows: {dup}" )
#     print("-" * 50)
#     print("\n" + "\n")

#     print("-" * 50)
#     print("Correlation Matrix (Numerical Columns):")
#     print("-" * 50)
#     # numdf = df.apply(pd.to_numeric, errors='coerce')
#     corr_matrix=numdf.corr()
#     print(corr_matrix )
#     print("\n" )

#     print("-" * 50)
#     # plt.figure(figsize=(8, 6))
#     # plt.imshow(corr_matrix, cmap='coolwarm', interpolation='nearest')
#     # plt.colorbar()
#     # plt.xticks(np.arange(len(corr_matrix.columns)), corr_matrix.columns, rotation=45, fontsize=10)
#     # plt.yticks(np.arange(len(corr_matrix.columns)), corr_matrix.columns, rotation=45, fontsize=10)
#     # plt.title('Correlation Matrix Heatmap', fontsize=14)
#     # plt.tight_layout()
#     # plt.show()

#     rows, cols = corr_matrix.shape

#     # Find the max column name length
#     max_label_length = max(len(label) for label in corr_matrix.columns)

#     # Dynamically adjust figure size based on column name length and number of columns
#     fig_width = max(8, cols * 0.5, max_label_length * 0.5)  # Scale width with column names
#     fig_height = max(6, rows * 0.5)  # Scale height with number of rows

#     plt.figure(figsize=(fig_width, fig_height))
#     plt.imshow(corr_matrix, cmap='coolwarm', interpolation='nearest')
#     plt.colorbar()

#     plt.xticks(np.arange(cols), corr_matrix.columns, rotation=45, ha="right", fontsize=12)
#     plt.yticks(np.arange(rows), corr_matrix.columns, rotation=0, fontsize=12)

#     plt.title('Correlation Matrix Heatmap', fontsize=14)
#     plt.tight_layout()
#     plt.show()
#     print("-" * 50 ) 
#     # print("", end="")
#     save=input('Save General Report? (Y/N):')
#     if save.lower()=="y":
#         Save_Genreal_Report(corr_matrix, dup, head, tail, des, null, info, shape)





def General_Analysis(df):
    print("\n" + "=" * 60)
    print("GENERAL ANALYSIS OF THE DATAFRAME".center(60))
    print("=" * 60 + "\n")

    # Adjust display settings for better output visibility
    pd.options.display.float_format = '{:,.2f}'.format
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 200)

    # 1. Shape of DataFrame
    shape = df.shape
    print("-" * 60)
    print(f"Shape of DataFrame: {shape}")
    print("-" * 60 + "\n")

    # 2. Column Information
    print("COLUMN INFORMATION".center(60, "-"))
    buffer = StringIO()
    df.info(buf=buffer)
    info = buffer.getvalue()
    print(info)
    print("\n")

    # 3. Missing Values
    print("MISSING VALUES IN EACH COLUMN".center(60, "-"))
    null_values = df.isnull().sum()
    print(null_values)
    print("\n")

    # 4. Descriptive Statistics (Numerical)
    numdf = df.select_dtypes(include=['number'])
    print("DESCRIPTIVE STATISTICS (NUMERICAL)".center(60, "-"))
    des = numdf.describe()
    print(tabulate(des, headers='keys', tablefmt='psql'))
    print("\n")

    # 5. First 5 Rows
    print("FIRST 5 ROWS OF THE DATAFRAME".center(60, "-"))
    head = df.head()
    print(tabulate(head, headers='keys', tablefmt='psql'))
    print("\n")

    # 6. Last 5 Rows
    print("LAST 5 ROWS OF THE DATAFRAME".center(60, "-"))
    tail = df.tail()
    print(tabulate(tail, headers='keys', tablefmt='psql'))
    print("\n")

    # 7. Duplicate Rows
    print("DUPLICATE ROWS".center(60, "-"))
    dup = df.duplicated().sum()
    print(f"Number of Duplicate Rows: {dup}")
    print("\n")

    # 8. Correlation Matrix
    print("CORRELATION MATRIX (NUMERICAL COLUMNS)".center(60, "-"))
    corr_matrix = numdf.corr()
    print(tabulate(corr_matrix, headers='keys', tablefmt='psql'))
    print("\n")

    # 9. Correlation Matrix Heatmap
    rows, cols = corr_matrix.shape
    max_label_length = max(len(label) for label in corr_matrix.columns)

    fig_width = max(8, cols * 0.5, max_label_length * 0.5)
    fig_height = max(6, rows * 0.5)

    plt.figure(figsize=(fig_width, fig_height))
    plt.imshow(corr_matrix, cmap='coolwarm', interpolation='nearest')
    plt.colorbar()
    plt.xticks(np.arange(cols), corr_matrix.columns, rotation=45, ha="right", fontsize=12)
    plt.yticks(np.arange(rows), corr_matrix.columns, rotation=0, fontsize=12)
    plt.title('Correlation Matrix Heatmap', fontsize=14)
    plt.tight_layout()
    plt.show()

    # 10. Save Report Option
    save = input('Save General Report? (Y/N): ')
    if save.lower() == "y":
        Save_Genreal_Report(corr_matrix, dup, head, tail, des, null_values, info, shape)



 
    


In [51]:
def Analyze_Data(df):
    b=True
    while(b):
        print("1. General Analysis")
        print("2. Custom Analysis")
        print("3. Return")
        choice=input("Enter Choice: ").strip()
        if(choice=="1"):
           General_Analysis(df)
        elif(choice=="2"):
            print(2)
            pass
        elif(choice=="3"):
          # print("Exiting Program...")
          b=False
        else:
          print("Please Enter Valid Choice !")

In [52]:
def Load_Data():
    b=True
    while(b):
        print("1. Analyze CSV File")
        print("2. Analyze Parquet File")
        print("3. Analyze Excel File")
        print("4. Return")
        choice=input("Enter Choice: ").strip()
        if(choice=="1"):
          path = input("Enter File Path of CSV File: ")
          try:
              df = pd.read_csv(path, encoding='unicode_escape')
              Analyze_Data(df)
          except (FileNotFoundError, Exception) as e:
              print(f"Error: {e}")
              print("CSV File not found. Try again !")
               
        elif(choice=="2"):
              path = input("Enter File Path of Parquet File: ")
              try:
                  df = pd.read_csv(path)
                  Analyze_Data(df)
              except (FileNotFoundError, Exception) as e:
                  print(f"Error: {e}")
                  print("Parquet File not found. Try again !")
        elif(choice=="3"):
              path = input("Enter File Path of Excel File: ")
              try:
                  df = pd.read_csv(path)
                  Analyze_Data(df)
              except (FileNotFoundError, Exception) as e:
                  print(f"Error: {e}")
                  print("Excel File not found. Try again !")
        elif(choice=="4"):
          # print("Exiting Program...")
          b=False
        else:
          print("Please Enter Valid Choice !")


In [53]:
def Edit_Data():
    b = True
    while b:
        print("1. Add a New Row")
        print("2. Update an Existing Row")
        print("3. Delete a Row")
        print("4. Edit Specific Column Data")
        print("5. Fill Missing Data")
        print("6. Rename Columns")
        print("7. Change Data Type of a Column")
        print("8. Sort Data")
        print("9. Filter Data")
        print("10. Remove Duplicates")
        print("11. Return")
        
        choice = input("Enter Choice: ")
        
        if choice == "1":
            print("Adding a new row...")
            pass
        elif choice == "2":
            print("Updating an existing row...")
            pass
        elif choice == "3":
            print("Deleting a row...")
            pass
        elif choice == "4":
            print("Editing specific column data...")
            pass
        elif choice == "5":
            print("Filling missing data...")
            pass
        elif choice == "6":
            print("Renaming columns...")
            pass
        elif choice == "7":
            print("Changing data type of a column...")
            pass
        elif choice == "8":
            print("Sorting data...")
            pass
        elif choice == "9":
            print("Filtering data...")
            pass
        elif choice == "10":
            print("Removing duplicates...")
            pass
        elif choice == "11":
            print("Returning to main menu...")
            b = False
        else:
            print("Please Enter a Valid Choice!")


In [ ]:
b=True
while(b):
    print("1. Analyze Data")
    print("2. Edit Data")
    print("3. Cleaning Data")
    print("4. Sort Folder")
    print("5. Exit")
    choice=input("Enter Choice: ")
    if(choice=="1"):
        Load_Data()
    elif(choice=="2"):
        print(2)
        pass
    elif(choice=="3"):
        print(3)
        pass
    elif(choice=="4"):
        print(4)
    elif(choice=="5"):
          print("Exiting Program...")
          b=False
    else:
        print("Please Enter Valid Choice !")
# C:\\Users\\Shivam\\Downloads\\Diwali Sales Data.csv
# C:\\Users\\Shivam\\Downloads\\road_accident_dataset.csv
# D:\\shivam
